##初始

In [ ]:
#挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')
####google云盘授权#####
##每个notebook执行一次###
__author__='CBA'
from google.colab import drive

#增加PyDrive操作库
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#授权登录
auth.authenticate_user()
gauth=GoogleAuth()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import print_function, division
import numpy as np
import tifffile as tiff
from PIL import Image
import codecs
import os
import cv2
import scipy.io as sio
from keras.utils.np_utils import to_categorical
from scipy.cluster.vq import whiten
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf


In [ ]:
#参数设置
r=7#patch半径
save_path='*/'#储存图片的位置

##函数

In [ ]:
#读取影像
def read_image(filename):
  img = tiff.imread(filename)
  img = np.asarray(img, dtype=np.float32)
  return img

In [ ]:
#归一化
def sample_normalization(data):
    """
    normalize each sample to 0-1
    Input:
        sample
    Output:
        Normalized sample
    """
    if np.max(data) == np.min(data):
        return np.ones_like(data, dtype=np.float32) * 1e-6
    else:
        return 1.0 * (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
#保存图片
def save_img_patch(img_patch, classes, i, j,save_path=save_path):
  print(img_patch.shape)
  img_patch_name = 'patch' + '_' + str(i) + '_' + str(j) +'_'+str(classes)+ '.png'
  img_patch_save_path = save_path + '/' + img_patch_name
  cv2.imwrite(img_patch_save_path, img_patch)
  return True

In [ ]:
#多分类切分训练样本patches
def create_patches_multiclasses(img_path,roi_img_path):
  s2_image=read_image(img_path)
  roi_image=read_image(roi_img_path)
  #维度调整
  # lst_img = lst_img.transpose(1,2,0)
  # train_img = train_img.transpose(1,2,0)
  #pad以防止roi出现在边界
  s2_image=np.pad(s2_image,((r,r),(r,r),(0,0)),'symmetric')
  #灰度图不能直接pad
  roi_image=cv2.cvtColor(roi_image, cv2.COLOR_GRAY2BGR)
  roi_image=np.pad(roi_image,((r,r),(r,r),(0,0)),'constant',constant_values=(0,0))
  roi_image=roi_image[:,:,0]
  #Normalization
  s2_image = sample_normalization(s2_image)
  s2_image = (s2_image - np.mean(s2_image))
  X=[]
  Y=[]
  # count=0
  #roi坐标打乱
  idx,idy=np.where(roi_image != 0)
  #ID = np.random.permutation(len(idx))#样本点打乱排序
  #idx=idx[ID]
  #idy=idy[ID]
  #莫名操作?
  # per=0.89
  # idx = idx[:int(per*len(idx))]
  # idy = idy[:int(per*len(idy))]
  print('获取roi_patches中')
  for i in tqdm(range(len(idx))):
    tmp_lst=s2_image[idx[i]-r:idx[i]+r+1,idy[i]-r:idy[i]+r+1,:]
    tmpy=roi_image[idx[i], idy[i]]
    if (tmpy != 0.0 ):
      X.append(tmp_lst)
      Y.append(int(tmpy))
    # count+=1
  X = np.asarray(X, dtype=np.float32)
  Y = np.asarray(Y, dtype=np.int8)
  return X,Y
#无标签样本patches
# def create_patches_unlabeled(img_path,roi_img_path):
#   s2_image=read_image(img_path)
#   roi_image=read_image(roi_img_path)
#   #维度调整
#   # lst_img = lst_img.transpose(1,2,0)
#   # train_img = train_img.transpose(1,2,0)
#   #pad以防止roi出现在边界
#   s2_image=np.pad(s2_image,((r,r),(r,r),(0,0)),'symmetric')
#   #灰度图不能直接pad
#   roi_image=cv2.cvtColor(roi_image, cv2.COLOR_GRAY2BGR)
#   roi_image=np.pad(roi_image,((r,r),(r,r),(0,0)),'constant',constant_values=(0,0))
#   roi_image=roi_image[:,:,0]
#   #Normalization
#   s2_image = sample_normalization(s2_image)
#   s2_image = (s2_image - np.mean(s2_image))
#   X=[]
#   Y=[]
#   # count=0
#   #roi坐标打乱
#   idx,idy=np.where(roi_image != 0)
#   #ID = np.random.permutation(len(idx))#样本点打乱排序
#   #idx=idx[ID]
#   #idy=idy[ID]
#   #莫名操作?
#   # per=0.89
#   # idx = idx[:int(per*len(idx))]
#   # idy = idy[:int(per*len(idy))]
#   print('获取roi_patches中')
#   for i in tqdm(range(len(idx))):
#     tmp_lst=s2_image[idx[i]-r:idx[i]+r+1,idy[i]-r:idy[i]+r+1,:]
#     tmpy=roi_image[idx[i], idy[i]]
#     if (tmpy != 0.0 ):
#       X.append(tmp_lst)
#       Y.append(int(tmpy))
#     # count+=1
#   X = np.asarray(X, dtype=np.float32)
#   Y = np.asarray(Y, dtype=np.int8)
#   return X,Y

##切patch


In [ ]:
#读入影像
s2_img_path = '/content/gdrive/MyDrive/CM/img/zhangbei_subset.tif'#替换为切patches的背景影像
roi_img_path='/content/gdrive/MyDrive/CM/RSL/RSL_image/test_roi_510.tif'#替换为标记的roi影像


In [ ]:
s2_img = read_image(s2_img_path)
roi_img = read_image(roi_img_path)

In [ ]:
#roi图padding防止边界roi点
roi_img=cv2.cvtColor(roi_img, cv2.COLOR_GRAY2BGR)
roi_img=np.pad(roi_img,((r,r),(r,r),(0,0)),'constant',constant_values=(0,0))
roi_img=roi_img[:,:,0]

In [ ]:
#创造样本池patches
X,Y=create_patches_multiclasses(s2_img_path,roi_img_path)

In [ ]:
#保存patches为numpy存储格式
np.save(X_train_path, X)
np.save(Y_train_path, Y)